1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변 생성 못 할수 있음
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("tax_with_table.docx")

document_list = loader.load_and_split(text_splitter)

In [2]:
len(document_list)


225

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
from langchain_upstage import UpstageEmbeddings
embedding = UpstageEmbeddings(model="embedding-query")

In [5]:
from langchain_chroma import Chroma

# # 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)


In [6]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# # `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
# retrieved_docs = database.similarity_search(query, k=3)

In [7]:
# retrieved_docs

In [19]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-mini")

In [20]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# [Question]
# {query}
# """

In [21]:
# ai_message = llm.invoke(prompt)

In [22]:
# ai_message.content

In [23]:
from langchain import hub
from langchain.chains import RetrievalQA

prompt = hub.pull("rlm/rag-prompt")
prompt

/opt/anaconda3/envs/project1122env/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [24]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [25]:
ai_message = qa_chain({"query": query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 4,194,000원입니다.'}